# 🚀 CodeReview AI - Complete Deployment

All code embedded - no file uploads needed!

## Features:
- 🔍 Code Review (6 focus options)
- ✨ Rewrite & Review
- 🔄 Code Translation (10+ languages)
- 👤 User Authentication
- 📊 Session Logging
- 🎨 Modern Dark Blue UI

---

## 📦 Step 1: Install Dependencies

In [8]:
!pip install -q flask flask-cors transformers torch accelerate sentencepiece protobuf pyngrok
print("✅ Dependencies installed!")

✅ Dependencies installed!


## 📁 Step 2: Create Project Structure

In [9]:
import os

os.makedirs('codereview_ai/templates', exist_ok=True)
os.chdir('codereview_ai')

print("✅ Project structure created!")
print(f"📂 Working directory: {os.getcwd()}")

✅ Project structure created!
📂 Working directory: /content/codereview_ai/codereview_ai


## 🐍 Step 3: Create app.py

In [10]:
%%writefile app.py
from flask import Flask, render_template, request, jsonify, session, redirect, url_for
from flask_cors import CORS
import sqlite3
import hashlib
import secrets
from datetime import datetime
import os
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import json

app = Flask(__name__)
app.secret_key = secrets.token_hex(32)
CORS(app)

# Database setup
def init_db():
    conn = sqlite3.connect('code_review.db')
    c = conn.cursor()

    c.execute('''CREATE TABLE IF NOT EXISTS users
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  username TEXT UNIQUE NOT NULL,
                  password TEXT NOT NULL,
                  created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP)''')

    c.execute('''CREATE TABLE IF NOT EXISTS user_sessions
                 (id INTEGER PRIMARY KEY AUTOINCREMENT,
                  user_id INTEGER,
                  login_time TIMESTAMP,
                  logout_time TIMESTAMP,
                  FOREIGN KEY(user_id) REFERENCES users(id))''')

    demo_password = hashlib.sha256("demo123".encode()).hexdigest()
    try:
        c.execute("INSERT INTO users (username, password) VALUES (?, ?)",
                 ("demo_user", demo_password))
    except sqlite3.IntegrityError:
        pass

    conn.commit()
    conn.close()
    print("✅ Database initialized!")

init_db()

MODEL_NAME = "Qwen/Qwen2.5-Coder-1.5B-Instruct"
print("📥 Loading AI model... (this may take 1-2 minutes)")

tokenizer = None
model = None

def load_model():
    global tokenizer, model
    if tokenizer is None or model is None:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
        model = AutoModelForCausalLM.from_pretrained(
            MODEL_NAME,
            torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
            device_map="auto"
        )
        print("✅ Model loaded successfully!")

def hash_password(password):
    return hashlib.sha256(password.encode()).hexdigest()

def get_db():
    conn = sqlite3.connect('code_review.db')
    conn.row_factory = sqlite3.Row
    return conn

def generate_response(prompt, max_length=2048):
    load_model()
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=4096)

    if torch.cuda.is_available():
        inputs = {k: v.to("cuda") for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_length,
            temperature=0.7,
            do_sample=True,
            top_p=0.95,
            pad_token_id=tokenizer.eos_token_id
        )

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if prompt in response:
        response = response.split(prompt)[-1].strip()

    return response

@app.route('/')
def index():
    if 'user_id' in session:
        return render_template('dashboard.html')
    return render_template('auth.html')

@app.route('/register', methods=['POST'])
def register():
    data = request.json
    username = data.get('username')
    password = data.get('password')

    if not username or not password:
        return jsonify({'success': False, 'message': 'Username and password required'})

    conn = get_db()
    try:
        hashed_pw = hash_password(password)
        conn.execute('INSERT INTO users (username, password) VALUES (?, ?)',
                    (username, hashed_pw))
        conn.commit()
        return jsonify({'success': True, 'message': 'Registration successful!'})
    except sqlite3.IntegrityError:
        return jsonify({'success': False, 'message': 'Username already exists'})
    finally:
        conn.close()

@app.route('/login', methods=['POST'])
def login():
    data = request.json
    username = data.get('username')
    password = data.get('password')

    conn = get_db()
    user = conn.execute('SELECT * FROM users WHERE username = ? AND password = ?',
                       (username, hash_password(password))).fetchone()

    if user:
        session['user_id'] = user['id']
        session['username'] = user['username']

        conn.execute('INSERT INTO user_sessions (user_id, login_time) VALUES (?, ?)',
                    (user['id'], datetime.now()))
        conn.commit()
        conn.close()

        return jsonify({'success': True, 'message': 'Login successful!'})

    conn.close()
    return jsonify({'success': False, 'message': 'Invalid credentials'})

@app.route('/logout', methods=['POST'])
def logout():
    if 'user_id' in session:
        conn = get_db()
        conn.execute('''UPDATE user_sessions
                       SET logout_time = ?
                       WHERE user_id = ? AND logout_time IS NULL
                       ORDER BY login_time DESC
                       LIMIT 1''',
                    (datetime.now(), session['user_id']))
        conn.commit()
        conn.close()
        session.clear()
    return jsonify({'success': True})

@app.route('/review-only', methods=['POST'])
def review_only():
    if 'user_id' not in session:
        return jsonify({'success': False, 'message': 'Not authenticated'})

    data = request.json
    code = data.get('code', '')
    language = data.get('language', 'python')
    focus = data.get('focus', 'general')

    focus_prompts = {
        'general': 'Provide a comprehensive code review',
        'performance': 'Focus on performance optimizations and efficiency',
        'security': 'Focus on security vulnerabilities and best practices',
        'bugs': 'Focus on identifying bugs and potential errors',
        'readability': 'Focus on code readability and maintainability',
        'best_practices': 'Focus on adherence to best practices and design patterns'
    }

    prompt = f"""You are an expert code reviewer. {focus_prompts.get(focus, focus_prompts['general'])}.

Language: {language}

Code to review:
```{language}
{code}
```

Provide a detailed analysis covering:
1. Code Quality Assessment
2. Potential Issues
3. Suggestions for Improvement
4. Educational Insights

Review:"""

    try:
        review = generate_response(prompt)
        return jsonify({'success': True, 'review': review})
    except Exception as e:
        return jsonify({'success': False, 'message': f'Error: {str(e)}'})

@app.route('/rewrite-review', methods=['POST'])
def rewrite_review():
    if 'user_id' not in session:
        return jsonify({'success': False, 'message': 'Not authenticated'})

    data = request.json
    code = data.get('code', '')
    language = data.get('language', 'python')

    prompt = f"""You are an expert code refactorer. Analyze the following code, provide a review, and then rewrite it with improvements.

Language: {language}

Original Code:
```{language}
{code}
```

Please provide:
1. A brief review of the original code
2. The improved/rewritten code
3. Explanation of changes made

Response:"""

    try:
        result = generate_response(prompt, max_length=3072)
        return jsonify({'success': True, 'result': result})
    except Exception as e:
        return jsonify({'success': False, 'message': f'Error: {str(e)}'})

@app.route('/translate', methods=['POST'])
def translate():
    if 'user_id' not in session:
        return jsonify({'success': False, 'message': 'Not authenticated'})

    data = request.json
    code = data.get('code', '')
    from_lang = data.get('from_language', 'python')
    to_lang = data.get('to_language', 'javascript')

    prompt = f"""You are an expert programmer. Translate the following code from {from_lang} to {to_lang}.
Maintain the same functionality and logic, but use idiomatic {to_lang} conventions.

Original {from_lang} code:
```{from_lang}
{code}
```

Translated {to_lang} code:
```{to_lang}"""

    try:
        translation = generate_response(prompt, max_length=3072)
        return jsonify({'success': True, 'translation': translation})
    except Exception as e:
        return jsonify({'success': False, 'message': f'Error: {str(e)}'})

@app.route('/dashboard')
def dashboard():
    if 'user_id' not in session:
        return redirect(url_for('index'))
    return render_template('dashboard.html')

if __name__ == '__main__':
    print("🚀 Starting Flask server on port 5000...")
    app.run(host='0.0.0.0', port=5000, debug=False)

Writing app.py


## 🎨 Step 4: Create auth.html

In [11]:
%%writefile templates/auth.html
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>CodeReview AI - Login</title>
    <link href="https://fonts.googleapis.com/css2?family=JetBrains+Mono:wght@400;600;700&family=Sora:wght@300;400;600;700&display=swap" rel="stylesheet">
    <style>
        * { margin: 0; padding: 0; box-sizing: border-box; }
        :root {
            --primary-blue: #0066FF; --deep-blue: #0A1628; --dark-blue: #0F1F3A;
            --mid-blue: #1A2F4F; --accent-cyan: #00D4FF; --text-primary: #E8F0FF;
            --text-secondary: #8FA8CC; --success: #00FF88; --error: #FF4466;
        }
        body {
            font-family: 'Sora', sans-serif;
            background: linear-gradient(135deg, var(--deep-blue) 0%, #000814 100%);
            color: var(--text-primary); min-height: 100vh; display: flex;
            align-items: center; justify-content: center; position: relative; overflow: hidden;
        }
        body::before {
            content: ''; position: absolute; width: 150%; height: 150%;
            background: radial-gradient(circle at 20% 30%, rgba(0, 102, 255, 0.15) 0%, transparent 50%),
                        radial-gradient(circle at 80% 70%, rgba(0, 212, 255, 0.1) 0%, transparent 50%);
            animation: pulse 8s ease-in-out infinite;
        }
        @keyframes pulse { 0%, 100% { transform: scale(1) rotate(0deg); } 50% { transform: scale(1.1) rotate(5deg); } }
        .auth-container { position: relative; z-index: 1; width: 90%; max-width: 450px; animation: slideUp 0.6s ease-out; }
        @keyframes slideUp { from { opacity: 0; transform: translateY(30px); } to { opacity: 1; transform: translateY(0); } }
        .logo-section { text-align: center; margin-bottom: 40px; }
        .logo {
            font-family: 'JetBrains Mono', monospace; font-size: 42px; font-weight: 700;
            background: linear-gradient(135deg, var(--primary-blue), var(--accent-cyan));
            -webkit-background-clip: text; -webkit-text-fill-color: transparent;
            text-shadow: 0 0 40px rgba(0, 102, 255, 0.5); letter-spacing: -1px;
        }
        .logo-subtitle { font-size: 14px; color: var(--text-secondary); margin-top: 8px; letter-spacing: 2px; text-transform: uppercase; font-weight: 300; }
        .auth-panel {
            background: rgba(15, 31, 58, 0.8); backdrop-filter: blur(20px);
            border: 1px solid rgba(0, 102, 255, 0.2); border-radius: 24px; padding: 40px;
            box-shadow: 0 20px 60px rgba(0, 0, 0, 0.5), 0 0 0 1px rgba(0, 212, 255, 0.1) inset;
        }
        .tab-switcher { display: flex; gap: 8px; margin-bottom: 32px; background: rgba(10, 22, 40, 0.6); padding: 6px; border-radius: 12px; }
        .tab-btn {
            flex: 1; padding: 12px; border: none; background: transparent; color: var(--text-secondary);
            font-family: 'Sora', sans-serif; font-size: 14px; font-weight: 600; cursor: pointer;
            border-radius: 8px; transition: all 0.3s ease; letter-spacing: 0.5px;
        }
        .tab-btn.active { background: linear-gradient(135deg, var(--primary-blue), #0052CC); color: white; box-shadow: 0 4px 12px rgba(0, 102, 255, 0.3); }
        .tab-content { display: none; animation: fadeIn 0.4s ease; }
        .tab-content.active { display: block; }
        @keyframes fadeIn { from { opacity: 0; transform: translateY(10px); } to { opacity: 1; transform: translateY(0); } }
        .form-group { margin-bottom: 24px; }
        label { display: block; font-size: 13px; font-weight: 600; color: var(--text-secondary); margin-bottom: 8px; letter-spacing: 0.5px; text-transform: uppercase; }
        input {
            width: 100%; padding: 14px 18px; background: rgba(10, 22, 40, 0.8);
            border: 1px solid rgba(0, 102, 255, 0.2); border-radius: 12px; color: var(--text-primary);
            font-family: 'JetBrains Mono', monospace; font-size: 14px; transition: all 0.3s ease;
        }
        input:focus { outline: none; border-color: var(--primary-blue); box-shadow: 0 0 0 3px rgba(0, 102, 255, 0.1); background: rgba(10, 22, 40, 1); }
        input::placeholder { color: var(--text-secondary); opacity: 0.5; }
        .submit-btn {
            width: 100%; padding: 16px; background: linear-gradient(135deg, var(--primary-blue), #0052CC);
            border: none; border-radius: 12px; color: white; font-family: 'Sora', sans-serif;
            font-size: 15px; font-weight: 600; cursor: pointer; transition: all 0.3s ease;
            letter-spacing: 0.5px; box-shadow: 0 4px 16px rgba(0, 102, 255, 0.3);
        }
        .submit-btn:hover { transform: translateY(-2px); box-shadow: 0 6px 24px rgba(0, 102, 255, 0.4); }
        .demo-info { margin-top: 24px; padding: 16px; background: rgba(0, 212, 255, 0.05); border: 1px solid rgba(0, 212, 255, 0.2); border-radius: 12px; text-align: center; }
        .demo-info-title { font-size: 12px; font-weight: 600; color: var(--accent-cyan); margin-bottom: 8px; letter-spacing: 1px; text-transform: uppercase; }
        .demo-credentials { font-family: 'JetBrains Mono', monospace; font-size: 13px; color: var(--text-secondary); line-height: 1.6; }
        .message { margin-top: 16px; padding: 12px; border-radius: 8px; font-size: 13px; text-align: center; animation: slideDown 0.3s ease; }
        @keyframes slideDown { from { opacity: 0; transform: translateY(-10px); } to { opacity: 1; transform: translateY(0); } }
        .message.success { background: rgba(0, 255, 136, 0.1); border: 1px solid rgba(0, 255, 136, 0.3); color: var(--success); }
        .message.error { background: rgba(255, 68, 102, 0.1); border: 1px solid rgba(255, 68, 102, 0.3); color: var(--error); }
    </style>
</head>
<body>
    <div class="auth-container">
        <div class="logo-section">
            <div class="logo">&lt;/&gt; CodeReview AI</div>
            <div class="logo-subtitle">Intelligent Code Analysis</div>
        </div>
        <div class="auth-panel">
            <div class="tab-switcher">
                <button class="tab-btn active" onclick="switchTab('login')">Login</button>
                <button class="tab-btn" onclick="switchTab('register')">Register</button>
            </div>
            <div id="login-tab" class="tab-content active">
                <form id="login-form">
                    <div class="form-group"><label for="login-username">Username</label><input type="text" id="login-username" placeholder="Enter username" required></div>
                    <div class="form-group"><label for="login-password">Password</label><input type="password" id="login-password" placeholder="Enter password" required></div>
                    <button type="submit" class="submit-btn">Sign In</button>
                    <div id="login-message"></div>
                </form>
                <div class="demo-info"><div class="demo-info-title">Demo Account</div><div class="demo-credentials">Username: <strong>demo_user</strong><br>Password: <strong>demo123</strong></div></div>
            </div>
            <div id="register-tab" class="tab-content">
                <form id="register-form">
                    <div class="form-group"><label for="register-username">Username</label><input type="text" id="register-username" placeholder="Choose username" required></div>
                    <div class="form-group"><label for="register-password">Password</label><input type="password" id="register-password" placeholder="Choose password" required></div>
                    <button type="submit" class="submit-btn">Create Account</button>
                    <div id="register-message"></div>
                </form>
            </div>
        </div>
    </div>
    <script>
        function switchTab(tab) { document.querySelectorAll('.tab-content').forEach(t => t.classList.remove('active')); document.querySelectorAll('.tab-btn').forEach(b => b.classList.remove('active')); document.getElementById(tab + '-tab').classList.add('active'); event.target.classList.add('active'); }
        function showMessage(formType, message, isSuccess) { const messageDiv = document.getElementById(formType + '-message'); messageDiv.textContent = message; messageDiv.className = 'message ' + (isSuccess ? 'success' : 'error'); setTimeout(() => { messageDiv.className = 'message'; messageDiv.textContent = ''; }, 3000); }
        document.getElementById('login-form').addEventListener('submit', async (e) => { e.preventDefault(); const username = document.getElementById('login-username').value; const password = document.getElementById('login-password').value; try { const response = await fetch('/login', { method: 'POST', headers: { 'Content-Type': 'application/json' }, body: JSON.stringify({ username, password }) }); const data = await response.json(); if (data.success) { showMessage('login', data.message, true); setTimeout(() => { window.location.href = '/dashboard'; }, 1000); } else { showMessage('login', data.message, false); } } catch (error) { showMessage('login', 'Connection error', false); } });
        document.getElementById('register-form').addEventListener('submit', async (e) => { e.preventDefault(); const username = document.getElementById('register-username').value; const password = document.getElementById('register-password').value; try { const response = await fetch('/register', { method: 'POST', headers: { 'Content-Type': 'application/json' }, body: JSON.stringify({ username, password }) }); const data = await response.json(); if (data.success) { showMessage('register', data.message, true); setTimeout(() => { switchTab('login'); document.querySelector('.tab-btn').click(); }, 1500); } else { showMessage('register', data.message, false); } } catch (error) { showMessage('register', 'Connection error', false); } });
    </script>
</body>
</html>

Writing templates/auth.html


## 📊 Step 5: Create dashboard.html

In [12]:
%%writefile templates/dashboard.html
<!DOCTYPE html>
<html><head><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1.0"><title>CodeReview AI - Dashboard</title><link href="https://fonts.googleapis.com/css2?family=JetBrains+Mono:wght@400;500;600;700&family=Sora:wght@300;400;500;600;700&display=swap" rel="stylesheet"><style>*{margin:0;padding:0;box-sizing:border-box}:root{--primary-blue:#0066FF;--deep-blue:#0A1628;--dark-blue:#0F1F3A;--mid-blue:#1A2F4F;--accent-cyan:#00D4FF;--text-primary:#E8F0FF;--text-secondary:#8FA8CC;--success:#00FF88;--error:#FF4466;--warning:#FFB800}body{font-family:'Sora',sans-serif;background:linear-gradient(135deg,var(--deep-blue) 0%,#000814 100%);color:var(--text-primary);min-height:100vh}.header{background:rgba(15,31,58,0.8);backdrop-filter:blur(20px);border-bottom:1px solid rgba(0,102,255,0.2);padding:20px 40px;display:flex;justify-content:space-between;align-items:center;box-shadow:0 4px 24px rgba(0,0,0,0.3)}.logo{font-family:'JetBrains Mono',monospace;font-size:24px;font-weight:700;background:linear-gradient(135deg,var(--primary-blue),var(--accent-cyan));-webkit-background-clip:text;-webkit-text-fill-color:transparent;display:flex;align-items:center;gap:12px}.logout-btn{padding:10px 24px;background:rgba(255,68,102,0.15);border:1px solid rgba(255,68,102,0.3);border-radius:8px;color:var(--error);font-family:'Sora',sans-serif;font-size:13px;font-weight:600;cursor:pointer;transition:all 0.3s ease}.logout-btn:hover{background:rgba(255,68,102,0.25);transform:translateY(-2px)}.container{max-width:1400px;margin:40px auto;padding:0 40px}.tab-nav{display:grid;grid-template-columns:repeat(auto-fit,minmax(200px,1fr));gap:16px;margin-bottom:40px}.tab-button{padding:20px;background:rgba(15,31,58,0.6);border:1px solid rgba(0,102,255,0.2);border-radius:16px;color:var(--text-secondary);font-family:'Sora',sans-serif;font-size:14px;font-weight:600;cursor:pointer;transition:all 0.3s ease;display:flex;flex-direction:column;align-items:center;gap:8px;text-align:center}.tab-button:hover{border-color:var(--primary-blue);transform:translateY(-4px);box-shadow:0 8px 24px rgba(0,102,255,0.2)}.tab-button.active{background:linear-gradient(135deg,rgba(0,102,255,0.3),rgba(0,82,204,0.3));border-color:var(--primary-blue);color:var(--text-primary);box-shadow:0 8px 32px rgba(0,102,255,0.3)}.tab-icon{font-size:32px}.tab-content{display:none;animation:fadeIn 0.4s ease}.tab-content.active{display:block}@keyframes fadeIn{from{opacity:0;transform:translateY(20px)}to{opacity:1;transform:translateY(0)}}.panel{background:rgba(15,31,58,0.6);border:1px solid rgba(0,102,255,0.2);border-radius:20px;padding:32px;box-shadow:0 12px 40px rgba(0,0,0,0.4)}.panel-title{font-size:24px;font-weight:700;margin-bottom:24px;background:linear-gradient(135deg,var(--text-primary),var(--accent-cyan));-webkit-background-clip:text;-webkit-text-fill-color:transparent}.form-grid{display:grid;gap:24px}.form-row{display:grid;grid-template-columns:repeat(auto-fit,minmax(200px,1fr));gap:16px}.form-group{display:flex;flex-direction:column;gap:8px}label{font-size:12px;font-weight:600;color:var(--text-secondary);letter-spacing:0.5px;text-transform:uppercase}select,textarea{padding:12px 16px;background:rgba(10,22,40,0.8);border:1px solid rgba(0,102,255,0.2);border-radius:12px;color:var(--text-primary);font-family:'JetBrains Mono',monospace;font-size:14px;transition:all 0.3s ease;resize:vertical}textarea{min-height:300px;line-height:1.6}select:focus,textarea:focus{outline:none;border-color:var(--primary-blue);box-shadow:0 0 0 3px rgba(0,102,255,0.1);background:rgba(10,22,40,1)}.submit-btn{padding:16px 32px;background:linear-gradient(135deg,var(--primary-blue),#0052CC);border:none;border-radius:12px;color:white;font-family:'Sora',sans-serif;font-size:15px;font-weight:600;cursor:pointer;transition:all 0.3s ease;box-shadow:0 4px 16px rgba(0,102,255,0.3)}.submit-btn:hover:not(:disabled){transform:translateY(-2px);box-shadow:0 6px 24px rgba(0,102,255,0.4)}.submit-btn:disabled{opacity:0.5;cursor:not-allowed}.result-section{margin-top:32px;padding:24px;background:rgba(10,22,40,0.6);border:1px solid rgba(0,212,255,0.2);border-radius:16px;display:none}.result-section.active{display:block;animation:slideUp 0.4s ease}@keyframes slideUp{from{opacity:0;transform:translateY(20px)}to{opacity:1;transform:translateY(0)}}.result-title{font-size:16px;font-weight:600;color:var(--accent-cyan);margin-bottom:16px;display:flex;align-items:center;gap:8px}.result-content{background:rgba(0,0,0,0.3);padding:20px;border-radius:12px;font-family:'JetBrains Mono',monospace;font-size:13px;line-height:1.8;white-space:pre-wrap;color:var(--text-primary);max-height:600px;overflow-y:auto}.loading{display:none;text-align:center;padding:40px}.loading.active{display:block}.spinner{width:50px;height:50px;border:4px solid rgba(0,102,255,0.2);border-top-color:var(--primary-blue);border-radius:50%;animation:spin 1s linear infinite;margin:0 auto 16px}@keyframes spin{to{transform:rotate(360deg)}}.loading-text{color:var(--text-secondary);font-size:14px}.focus-options{display:grid;grid-template-columns:repeat(auto-fit,minmax(150px,1fr));gap:12px}.focus-btn{padding:12px 16px;background:rgba(10,22,40,0.6);border:1px solid rgba(0,102,255,0.2);border-radius:10px;color:var(--text-secondary);font-family:'Sora',sans-serif;font-size:13px;font-weight:500;cursor:pointer;transition:all 0.3s ease}.focus-btn:hover{border-color:var(--primary-blue);background:rgba(0,102,255,0.1)}.focus-btn.active{background:linear-gradient(135deg,rgba(0,102,255,0.3),rgba(0,82,204,0.3));border-color:var(--primary-blue);color:var(--text-primary)}</style></head><body><div class="header"><div class="logo"><span>&lt;/&gt;</span><span>CodeReview AI</span></div><button class="logout-btn" onclick="logout()">Logout</button></div><div class="container"><div class="tab-nav"><button class="tab-button active" onclick="switchTab('review-only')"><span class="tab-icon">🔍</span><span>Review Only</span></button><button class="tab-button" onclick="switchTab('rewrite')"><span class="tab-icon">✨</span><span>Rewrite & Review</span></button><button class="tab-button" onclick="switchTab('translate')"><span class="tab-icon">🔄</span><span>Code Translation</span></button></div><div id="review-only-content" class="tab-content active"><div class="panel"><h2 class="panel-title">Code Review with Focus Options</h2><form id="review-form" class="form-grid"><div class="form-row"><div class="form-group"><label>Programming Language</label><select id="review-language"><option value="python">Python</option><option value="javascript">JavaScript</option><option value="java">Java</option><option value="cpp">C++</option><option value="csharp">C#</option><option value="go">Go</option><option value="rust">Rust</option><option value="php">PHP</option><option value="ruby">Ruby</option><option value="typescript">TypeScript</option></select></div></div><div class="form-group"><label>Review Focus</label><div class="focus-options"><button type="button" class="focus-btn active" data-focus="general">General</button><button type="button" class="focus-btn" data-focus="performance">Performance</button><button type="button" class="focus-btn" data-focus="security">Security</button><button type="button" class="focus-btn" data-focus="bugs">Bugs</button><button type="button" class="focus-btn" data-focus="readability">Readability</button><button type="button" class="focus-btn" data-focus="best_practices">Best Practices</button></div></div><div class="form-group"><label>Code to Review</label><textarea id="review-code" placeholder="Paste your code here..."></textarea></div><button type="submit" class="submit-btn">🔍 Analyze Code</button></form><div class="loading" id="review-loading"><div class="spinner"></div><div class="loading-text">Analyzing your code...</div></div><div class="result-section" id="review-result"><div class="result-title">📊 Code Review Analysis</div><div class="result-content" id="review-output"></div></div></div></div><div id="rewrite-content" class="tab-content"><div class="panel"><h2 class="panel-title">Rewrite & Review Code</h2><form id="rewrite-form" class="form-grid"><div class="form-row"><div class="form-group"><label>Programming Language</label><select id="rewrite-language"><option value="python">Python</option><option value="javascript">JavaScript</option><option value="java">Java</option><option value="cpp">C++</option><option value="csharp">C#</option><option value="go">Go</option><option value="rust">Rust</option><option value="php">PHP</option><option value="ruby">Ruby</option><option value="typescript">TypeScript</option></select></div></div><div class="form-group"><label>Original Code</label><textarea id="rewrite-code" placeholder="Paste your code here..."></textarea></div><button type="submit" class="submit-btn">✨ Rewrite & Review</button></form><div class="loading" id="rewrite-loading"><div class="spinner"></div><div class="loading-text">Rewriting your code...</div></div><div class="result-section" id="rewrite-result"><div class="result-title">✨ Improved Code & Analysis</div><div class="result-content" id="rewrite-output"></div></div></div></div><div id="translate-content" class="tab-content"><div class="panel"><h2 class="panel-title">Code Translation</h2><form id="translate-form" class="form-grid"><div class="form-row"><div class="form-group"><label>From Language</label><select id="from-language"><option value="python">Python</option><option value="javascript">JavaScript</option><option value="java">Java</option><option value="cpp">C++</option><option value="csharp">C#</option><option value="go">Go</option><option value="rust">Rust</option><option value="php">PHP</option><option value="ruby">Ruby</option><option value="typescript">TypeScript</option></select></div><div class="form-group"><label>To Language</label><select id="to-language"><option value="javascript">JavaScript</option><option value="python">Python</option><option value="java">Java</option><option value="cpp">C++</option><option value="csharp">C#</option><option value="go">Go</option><option value="rust">Rust</option><option value="php">PHP</option><option value="ruby">Ruby</option><option value="typescript">TypeScript</option></select></div></div><div class="form-group"><label>Code to Translate</label><textarea id="translate-code" placeholder="Paste your code here..."></textarea></div><button type="submit" class="submit-btn">🔄 Translate Code</button></form><div class="loading" id="translate-loading"><div class="spinner"></div><div class="loading-text">Translating your code...</div></div><div class="result-section" id="translate-result"><div class="result-title">🔄 Translated Code</div><div class="result-content" id="translate-output"></div></div></div></div></div><script>function switchTab(t){document.querySelectorAll('.tab-content').forEach(e=>e.classList.remove('active'));document.querySelectorAll('.tab-button').forEach(e=>e.classList.remove('active'));document.getElementById(t+'-content').classList.add('active');event.target.closest('.tab-button').classList.add('active')}async function logout(){await fetch('/logout',{method:'POST'});window.location.href='/'}document.querySelectorAll('.focus-btn').forEach(e=>{e.addEventListener('click',function(){document.querySelectorAll('.focus-btn').forEach(e=>e.classList.remove('active'));this.classList.add('active')})});document.getElementById('review-form').addEventListener('submit',async e=>{e.preventDefault();const t=document.getElementById('review-code').value,n=document.getElementById('review-language').value,o=document.querySelector('.focus-btn.active').dataset.focus;if(!t.trim()){alert('Please enter code to review');return}const a=document.getElementById('review-loading'),c=document.getElementById('review-result'),s=e.target.querySelector('.submit-btn');a.classList.add('active');c.classList.remove('active');s.disabled=!0;try{const e=await fetch('/review-only',{method:'POST',headers:{'Content-Type':'application/json'},body:JSON.stringify({code:t,language:n,focus:o})}),r=await e.json();r.success?(document.getElementById('review-output').textContent=r.review,c.classList.add('active')):alert('Error: '+r.message)}catch(e){alert('Connection error')}finally{a.classList.remove('active');s.disabled=!1}});document.getElementById('rewrite-form').addEventListener('submit',async e=>{e.preventDefault();const t=document.getElementById('rewrite-code').value,n=document.getElementById('rewrite-language').value;if(!t.trim()){alert('Please enter code to rewrite');return}const o=document.getElementById('rewrite-loading'),a=document.getElementById('rewrite-result'),c=e.target.querySelector('.submit-btn');o.classList.add('active');a.classList.remove('active');c.disabled=!0;try{const e=await fetch('/rewrite-review',{method:'POST',headers:{'Content-Type':'application/json'},body:JSON.stringify({code:t,language:n})}),r=await e.json();r.success?(document.getElementById('rewrite-output').textContent=r.result,a.classList.add('active')):alert('Error: '+r.message)}catch(e){alert('Connection error')}finally{o.classList.remove('active');c.disabled=!1}});document.getElementById('translate-form').addEventListener('submit',async e=>{e.preventDefault();const t=document.getElementById('translate-code').value,n=document.getElementById('from-language').value,o=document.getElementById('to-language').value;if(!t.trim()){alert('Please enter code to translate');return}if(n===o){alert('Please select different languages');return}const a=document.getElementById('translate-loading'),c=document.getElementById('translate-result'),s=e.target.querySelector('.submit-btn');a.classList.add('active');c.classList.remove('active');s.disabled=!0;try{const e=await fetch('/translate',{method:'POST',headers:{'Content-Type':'application/json'},body:JSON.stringify({code:t,from_language:n,to_language:o})}),r=await e.json();r.success?(document.getElementById('translate-output').textContent=r.translation,c.classList.add('active')):alert('Error: '+r.message)}catch(e){alert('Connection error')}finally{a.classList.remove('active');s.disabled=!1}});</script></body></html>

Writing templates/dashboard.html


## 🔑 Step 6: Configure ngrok

Get your free auth token: https://dashboard.ngrok.com/get-started/your-authtoken

In [13]:
from pyngrok import ngrok
import getpass

auth_token = getpass.getpass('🔑 Enter your ngrok auth token: ')
ngrok.set_auth_token(auth_token)

print("✅ ngrok configured!")

🔑 Enter your ngrok auth token: ··········
✅ ngrok configured!


## 🚀 Step 7: Launch Application

In [14]:
import threading
import time
from pyngrok import ngrok

def run_flask():
    !python app.py

flask_thread = threading.Thread(target=run_flask)
flask_thread.daemon = True
flask_thread.start()

print("⏳ Starting server and loading AI model...")
print("⚠️  First run may take 1-2 minutes...")
time.sleep(25)  # Give extra time for database and model initialization

public_url = ngrok.connect(5000, bind_tls=True)

print("\n" + "="*70)
print("✅ CODEREVIEW AI IS LIVE!")
print("="*70)
print(f"\n🌐 URL: {public_url}")
print("\n👤 Demo Login:")
print("   Username: demo_user")
print("   Password: demo123")
print("\n📊 Features:")
print("   ✓ Code Review (6 focus options)")
print("   ✓ Rewrite & Review")
print("   ✓ Code Translation")
print("   ✓ Session Logging")
print("\n⚠️  Keep this notebook running!")
print("="*70 + "\n")

⏳ Starting server and loading AI model...
⚠️  First run may take 1-2 minutes...
✅ Database initialized!
📥 Loading AI model... (this may take 1-2 minutes)
🚀 Starting Flask server on port 5000...
 * Serving Flask app 'app'
 * Debug mode: off
Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.

✅ CODEREVIEW AI IS LIVE!

🌐 URL: NgrokTunnel: "https://timothy-uncircularized-josh.ngrok-free.dev" -> "http://localhost:5000"

👤 Demo Login:
   Username: demo_user
   Password: demo123

📊 Features:
   ✓ Code Review (6 focus options)
   ✓ Rewrite & Review
   ✓ Code Translation
   ✓ Session Logging

⚠️  Keep this notebook running!



## 📊 Step 8: View Session Logs (Run AFTER logging in)

In [15]:
import sqlite3
import pandas as pd
import os

# Check if database exists
if not os.path.exists('code_review.db'):
    print("❌ Database not found! Make sure the Flask app is running.")
else:
    conn = sqlite3.connect('code_review.db')

    print("👥 Registered Users:")
    print("="*50)
    users = pd.read_sql_query("SELECT id, username, created_at FROM users", conn)
    print(users)

    print("\n📝 User Sessions:")
    print("="*50)
    try:
        sessions = pd.read_sql_query("""
            SELECT us.id, u.username, us.login_time, us.logout_time
            FROM user_sessions us
            JOIN users u ON us.user_id = u.id
            ORDER BY us.login_time DESC
            LIMIT 10
        """, conn)
        if len(sessions) > 0:
            print(sessions)
        else:
            print("No sessions yet. Login to the app to create sessions!")
    except Exception as e:
        print(f"No sessions yet: {e}")

    conn.close()

👥 Registered Users:
   id   username           created_at
0   1  demo_user  2026-01-31 05:10:54

📝 User Sessions:
No sessions yet. Login to the app to create sessions!


In [16]:
git remote add origin https://github.com/Hrithik-balaji/CodeRefiner.git
git branch -M main
git push -u origin main

SyntaxError: invalid syntax (ipython-input-2617324748.py, line 1)